In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import datetime

In [ ]:
req = requests.get('http://www.twtc.com.tw/exhibition.aspx?p=home')
soup = bs(req.text, 'html5lib')

In [ ]:
fromData = {
    # 今年
    'ctl00$Body$ddlYear': datetime.datetime.now().year,
    # 第幾季
    'ctl00$Body$ddlMoth': datetime.datetime.now().month//4+1
}
for f in soup.select('input[type="hidden"]'):
    try:
        fromData[f['name']] = f['value']
    except KeyError:
        pass
dateTime = []
title = []
urlList = []
local = []
for n in range(datetime.datetime.now().month//4+1, 5):
    fromData['ctl00$Body$ddlMoth'] = n
    req2 = requests.post('http://www.twtc.com.tw/exhibition.aspx?p=home', data=fromData)
    soup2 = bs(req2.text, 'html5lib')
    for tr in soup2.find('table').findAll('tr'):
        if tr.select('td'):
            dateTime.append(tr.select('td')[0].text)
            title.append(tr.select('td')[1].text[:-4].strip())
            urlList.append(tr.select('td')[1].find('a')['href'])
            local.append(tr.select('td')[4].text)
    
    

In [ ]:
dData = {'dateTime':dateTime,
        'title':title,
        'local':local,
        'urlList':urlList}

In [ ]:
DataDF = pd.DataFrame(dData)

In [ ]:
DataDF['datetimeStart'] = str(datetime.datetime.now().year) + '/' + DataDF['dateTime'].str.split(' ~ ',expand=True)[0]
DataDF['datetimeEnd'] = str(datetime.datetime.now().year) + '/' + DataDF['dateTime'].str.split(' ~ ',expand=True)[1]
DataDF['type'] = 'twtc'


In [ ]:
DataDF[['title', 'datetimeStart', 'datetimeEnd', 'local', 'urlList', 'type']]